In [1]:
from astropy.utils.data import download_file
from astropy.io import ascii
from astropy.table import Table
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
def get_confirmed_planets(cache=True, show_progress=True,
                            select=None):
    """
    Download (and optionally cache) a table from the `NExScI Exoplanet Archive 
                                <http://exoplanetarchive.ipac.caltech.edu/index.html>`_.
    Based on the code from `astroquery <https://github.com/astropy/astroquery>`_.
                                
    Parameters
    ----------
    cache : bool (optional)
        Cache table to local astropy cache? Default is `True`.
    show_progress : bool (optional)
        Show progress of table download (if no cached copy is
        available). Default is `True`.
    select : str (optional)
        Comma-separated, no spaces string indicating columns to be
        returned. Default `None` will select all default columns
        as set by the Exoplanet Archive API. `*` will select all.
    Returns
    -------
    table : `~astropy.table`
        Astropy table of requested data.
    """
    url = 'http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets'
    url += '&format=csv'
    if not select is None:
        url += '&select='+select
    table_path = download_file(url, cache=cache,
                                   show_progress=show_progress,
                                   timeout=120)
    table = ascii.read(table_path)
    
    # Store column of planet names:
    pl_names = [host_name.replace(' ', '') + letter
                       for host_name, letter in
                       zip(table['pl_hostname'].data,
                           table['pl_letter'].data)]
    table['pl_name'] = pl_names
    
    return table

In [3]:
def get_cepheids():
    """
    Execute ADQL query for Gaia DR1 Cepheids using astroquery's TAP+ utils.
    
    Returns
    -------
    table : `~astropy.table`
        Astropy table of requested data.
    """
    job = Gaia.launch_job("SELECT cep.*, gaia.ra, gaia.dec \
        FROM gaiadr1.cepheid AS cep \
        INNER JOIN gaiadr1.gaia_source as gaia \
        on cep.source_id = gaia.source_id")
    return job.get_results()

In [4]:
import astropy.units as u
from astropy.coordinates import SkyCoord
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
r.pprint()

Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
0.0026034636994048854 1635721458409799680 ...                   --
0.0038518741347606357 1635721458409799680 ...                   --
0.0045454265009678297 1635721458409799680 ...                   --
 0.005613919443965546 1635721458409799680 ...                   --
0.0058464347158221214 1635721458409799680 ...                   --
0.0062090426663719287 1635721458409799680 ...                   --
 0.007469463683838576 1635721458409799680 ...                   --
0.0082020045145243161 1635721458409799680 ...                   --
0.0083385096908740273 1635721458409799680 ...                   --
0.0084066777722589213 1635721458409799680 ...                   --
                  ...                 ... ...                  ...
 0.049718018073992835 1635721458409799680 ... 

In [ ]:
job = Gaia.launch_job("select top 100 \
solution_id,ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations,duplicated_source,phot_variable_flag \
from gaiadr1.gaia_source order by source_id")
print(job)

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

In [ ]:
job = Gaia.launch_job_async("SELECT * \
FROM gaiadr1.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr1.gaia_source.ra,gaiadr1.gaia_source.dec),CIRCLE('ICRS',56.75,24.1167,2))=1;" \
, dump_to_file=False)

print (job)